In [ ]:
%reset -f
%matplotlib inline
import random
import torch

In [ ]:
def create_data(w, b, num_examples):
    """生成 y = Xw + b +噪声  """
    x = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(x, w) + b
    y += torch.normal(0, 0.01, y.shape)

    return x, y.reshape((-1, 1))

In [ ]:
def data_iter(batch_size, feature, labels):
    num_examples = len(feature)
    indices = list(range(num_examples))
    random.shuffle(indices)
   # print("data_iter")
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(indices[i:min(i+batch_size, num_examples)])
        #print("i ", i)
        #print(batch_indices)
        yield feature[batch_indices], labels[batch_indices]

In [ ]:
def linreg(X, w, b):
    """"线性回归模型"""
    return torch.matmul(X, w) + b

In [ ]:
def squared_loss(y_hat, y):
    """均方损失"""
    return (y_hat - y.reshape(y_hat.shape))**2 / 2

In [ ]:
def sgd(params, lr, batch_size):
    """小批量随机梯度下降"""
    with torch.no_grad():
        for param in params:
            param -= lr*param.grad/batch_size
            param.grad.zero_()
            #print("param = ",param)

In [ ]:
lr = 0.03
num_epochs = 3
batch_size = 10

true_w = torch.tensor([2, -3.4])
true_b = 4.2

train_w = torch.normal(0, 0.01, size = (2, 1), requires_grad = True)
train_b = torch.zeros(1, requires_grad = True)

features, labels = create_data(true_w, true_b, 1000)

In [ ]:
print("train_w = ", train_w)
print("train_b = ", train_b)

In [ ]:
features.shape, features

In [ ]:
for X, y in data_iter(batch_size, features, labels):
    print(X,'\n', y)
    break

In [ ]:
for eporch in range(num_epochs):
    j = 0
    for X, y in data_iter(batch_size, features, labels):
       # print("jjjj")
        l = squared_loss(linreg(X, train_w, train_b), y)
        l.sum().backward()
        sgd([train_w, train_b], lr, batch_size)
        j = j + 1
        #print("j ",j)

    with torch.no_grad():
        train_l = squared_loss(linreg(features, train_w, train_b), labels)
        print(f'epoch B{eporch + 1}, loss {float(train_l.mean()):f}')

In [ ]:
print(train_w)
print(train_b)

In [ ]:
print(true_w)
print(true_b)

In [ ]:
print(f'w的估计误差: {true_w - train_w.reshape(true_w.shape)}')
print(f'b的估计误差: {true_b - train_b}')